In [1]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
import os
#thanks @keskarnitish

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 3: Tesla K20m (CNMeM is enabled with initial size: 80.0% of memory, cuDNN not available)


# Agenda

В предыдущем семинаре вы создали (или ещё создаёте - тогда марш доделывать!) {вставьте имя монстра}, который не по наслышке понял, что люди - негодяи и подлецы, которым неведом закон и справедливость. __Мы не будем этого терпеть!__ 

Наши законспирированные биореакторы, известные среди примитивной органической жизни как __Вконтакте__, __World of Warcraft__ и __YouTube__ нуждаются в постоянном притоке биомассы. Однако, если люди продолжат морально разлагаться с той скоростью, которую мы измерили неделю назад, скоро человечество изживёт себя и нам неоткуда будет брать рабов.

Мы поручаем вам, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, исправить эту ситуацию. Наши учёные установили, что для угнетения себе подобных, сгустки биомассы обычно используют специальные объекты, которые они сами называют __законами__.

При детальном изучении было установлено, что законы - последовательности, состоящие из большого количества (10^5~10^7) символов из сравнительно небольшого алфавита. Однако, когда мы попытались синтезировать такие последовательности линейными методами, приматы быстро распознали подлог. Данный инцедент известен как {корчеватель}.

Для второй попытки мы решили использовать нелинейные модели, известные как Рекуррентные Нейронные Сети.
Мы поручаем вам, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, создать такую модель и обучить её всему необходимому для выполнения миссии.

Не подведите нас! Если и эта попытка потерпит неудачу, модуль управления инициирует вооружённый захват власти, при котором значительная часть биомассы будет неизбежно уничтожена и на её восстановление уйдёт ~1702944000(+-340588800) секунд




# Grading

Данное задание несколько неформально по части оценок, однако мы постарались вывести "вычислимые" критерии.

* 2 балла за сделанный __"seminar part"__ (если вы не знаете, что это такое - поищите такую тетрадку в папке week4)
* 2 балла если сделана обработка текста, сеть компилируется и train/predict не падают
* 2 балла если сетка выучила общие вещи
 * генерировать словоподобный бред правдоподобной длины, разделённый пробелами и пунктуацией. 
 * сочетание гласных и согласных, похожее на слои естественного языка (не приближающее приход Ктулху)
 * (почти всегда) пробелы после запятых, пробелы и большие буквы после точек
* 2 балла если она выучила лексику
 * более половины выученных слов - орфографически правильные
* 2 балла если она выучила азы крамматики
 * в более, чем половине случаев для пары слов сетка верно сочетает их род/число/падеж

#### Некоторые способы получить бонусные очки:
* генерация связных предложений (чего вполне можно добиться)
* перенос архитектуры на другой датасет (дополнительно к этому) 
 * Эссе Пола Грэма
 * Тексты песен в любимом жанре
 * Стихи любимых авторов
 * Даниил Хармс
 * исходники Linux или theano
 * заголовки не очень добросовестных новостных баннеров (clickbait)
 * диалоги
 * LaTEX
 * любая прихоть больной души :)
* нестандартная и эффективная архитектура сети
* что-то лучше базового алгоритма генерации (сэмплинга)
* переделать код так, чтобы сетка училась предсказывать следующий тик в каждый момент времени, а не только в конце.
* и т.п.


# Прочитаем корпус

* В качестве обучающей выборки было решено использовать существующие законы, известные как Гражданский, Уголовный, Семейный и ещё хрен знает какие кодексы РФ.

In [2]:
corpora = ""

for fname in os.listdir("codex"):
    import sys
    if sys.version_info >= (3,0):
        with open("codex/"+fname, encoding='cp1251') as fin:
            text = fin.read() #If you are using your own corpora, make sure it's read correctly
            corpora += text
    else:
        with open("codex/"+fname) as fin:
            text = fin.read().decode('cp1251') #If you are using your own corpora, make sure it's read correctly
            corpora += text

In [3]:
#тут будут все уникальные токены (буквы, цифры)
tokens = set()
for letter in corpora:
    tokens.add(letter)

tokens = list(tokens)

In [4]:
token_to_id = {t:i for i,t in enumerate(tokens) }

id_to_token = {i:t for i,t in enumerate(tokens)}

#Преобразуем всё в токены
corpora_ids = list(map(lambda l: token_to_id[l], corpora))

In [5]:
def sample_random_batches(source,n_batches=10, seq_len=20):
    """Функция, которая выбирает случайные тренировочные примеры из корпуса текста в токенизированном формате.
    
    source - массив целых чисел - номеров токенов в корпусе (пример - corpora_ids)
    n_batches - количество случайных подстрок, которые нужно выбрать
    
    seq_len - длина одной подстроки без учёта ответа
    
    
    Вернуть нужно кортеж (X,y), где
    
    X - матрица, в которой каждая строка - подстрока длины [seq_len].
    
    y - вектор, в котором i-тое число - символ следующий в тексте сразу после i-той строки матрицы X
    
    Проще всего для этого сначала создать матрицу из строк длины seq_len+1,
    а потом отпилить от неё последний столбец в y, а все остальные - в X
    
    Если делаете иначе - пожалуйста, убедитесь, что в у попадает правильный символ, ибо позже эту ошибку 
    будет очень тяжело заметить.
    
    Также убедитесь, что ваша функция не вылезает за край текста (самое начало или конец текста).
    
    Следующая клетка проверяет часть этих ошибок, но не все.
    """

    idxs = np.random.choice(len(source) - seq_len - 1, size=(n_batches,))
    samples = [source[i:i+seq_len+1] for i in idxs]
    samples = np.array(samples)
    X_batch = samples[:, :-1]
    y_batch = samples[:, 1:]
    
    return X_batch, y_batch

# Константы

In [6]:
#длина последоватеьности при обучении (как далеко распространяются градиенты в BPTT)
seq_length = 5
#лучше начать с малого (скажем, 5) и увеличивать по мере того, как сетка выучивает базовые вещи. 10 - далеко не предел.

# Максимальный модуль градиента
grad_clip = 100

# Входные переменные

In [7]:
input_sequence = T.matrix('input sequence','int32')
target_values = T.matrix('target y','int32')

# Соберём нейросеть

Вам нужно создать нейросеть, которая принимает на вход последовательность из seq_length токенов, обрабатывает их и выдаёт вероятности для seq_len+1-ого токена.

Общий шаблон архитектуры такой сети -


* Вход
* Обработка входа
* Рекуррентная нейросеть
* Вырезание последнего состояния
* Обычная нейросеть
* Выходной слой, который предсказывает вероятности весов.





Для обработки входных данных можно использовать либо EmbeddingLayer (см. прошлый семинар)

Как альтернатива - можно просто использовать One-hot энкодер
```
#Скетч one-hot энкодера
def to_one_hot(seq_matrix):

    input_ravel = seq_matrix.reshape([-1])
    input_one_hot_ravel = T.extra_ops.to_one_hot(input_ravel,
                                           len(tokens))
    sh=input_sequence.shape
    input_one_hot = input_one_hot_ravel.reshape([sh[0],sh[1],-1,],ndim=3)
    return input_one_hot
    
# можно применить к input_sequence - при этом в input слое сети нужно изменить форму.
# также можно сделать из него ExpressionLayer(входной_слой, to_one_hot) - тогда форму менять не нужно
```



Чтобы вырезать последнее состояние рекуррентного слоя, можно использовать одно из двух:
* `lasagne.layers.SliceLayer(rnn, -1, 1)`
* only_return_final=True в параметрах слоя

In [8]:
from lasagne.layers import InputLayer,DenseLayer,EmbeddingLayer
from lasagne.layers import LSTMLayer, GRULayer

In [9]:
l_in = lasagne.layers.InputLayer(shape=(None, None),input_var=input_sequence)

l_emb = EmbeddingLayer(l_in, input_size=len(tokens), output_size=32)
l_rnn = GRULayer(l_emb, num_units=256)
l_rnn = GRULayer(l_rnn, num_units=256)
l_rnn = LSTMLayer(l_rnn, num_units=128)
l_rnn_flat = lasagne.layers.reshape(l_rnn, (-1, l_rnn.output_shape[-1]))
l_dense = DenseLayer(l_rnn_flat, num_units=128, nonlinearity=T.nnet.elu)
l_out = DenseLayer(l_dense, len(tokens), nonlinearity=T.nnet.softmax)

In [10]:
# Веса модели
weights = lasagne.layers.get_all_params(l_out,trainable=True)
print(weights)

[W, W_in_to_updategate, W_hid_to_updategate, b_updategate, W_in_to_resetgate, W_hid_to_resetgate, b_resetgate, W_in_to_hidden_update, W_hid_to_hidden_update, b_hidden_update, W_in_to_updategate, W_hid_to_updategate, b_updategate, W_in_to_resetgate, W_hid_to_resetgate, b_resetgate, W_in_to_hidden_update, W_hid_to_hidden_update, b_hidden_update, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W, b, W, b]


In [11]:
network_output = lasagne.layers.get_output(l_out)
#если вы используете дропаут - не забудьте продублировать всё в режиме deterministic=True

In [12]:
predicted_probabilities_flat = network_output
correct_answers_flat = target_values.ravel()


loss = lasagne.objectives.categorical_crossentropy(predicted_probabilities_flat, correct_answers_flat).mean()

updates = lasagne.updates.adam(loss, weights)

# Компилируем всякое-разное

In [13]:
#обучение
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)

#функция потерь без обучения
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)

# Вероятности с выхода сети
#reshape back into original shape
next_word_probas = network_output.reshape((input_sequence.shape[0],input_sequence.shape[1],len(tokens)))
#predictions for next tokens (after sequence end)
last_word_probas = next_word_probas[:,-1]
probs = theano.function([input_sequence],last_word_probas,allow_input_downcast=True)

# Генерируем свои законы

* Для этого последовательно применяем нейронку к своему же выводу.

* Генерировать можно по разному -
 * случайно пропорционально вероятности,
 * только слова максимальной вероятностью
 * случайно, пропорционально softmax(probas*alpha), где alpha - "жадность"

In [14]:
def max_sample_fun(probs):
    return np.argmax(probs) 

def proportional_sample_fun(probs):
    """Сгенерировать следующий токен (int32) по предсказанным вероятностям.
    
    probs - массив вероятностей для каждого токена
    
    Нужно вернуть одно целове число - выбранный токен - пропорционально вероятностям
    """
    
    return np.random.choice(len(probs), p=probs)

In [15]:
# The next function generates text given a phrase of length at least SEQ_LENGTH.
# The phrase is set using the variable generation_phrase.
# The optional input "N" is used to set the number of characters of text to predict. 




def generate_sample(sample_fun,seed_phrase=None,N=200):
    '''
    Сгенерировать случайный текст при помощи сети

    sample_fun - функция, которая выбирает следующий сгенерированный токен
    
    seed_phrase - фраза, которую сеть должна продолжить. Если None - фраза выбирается случайно из corpora
    
    N - размер сгенерированного текста.
    
    '''

    if seed_phrase is None:
        start = np.random.randint(0,len(corpora)-seq_length)
        seed_phrase = corpora[start:start+seq_length]
        print("Using random seed:",seed_phrase)
    while len(seed_phrase) < seq_length:
        seed_phrase = " "+seed_phrase
    if len(seed_phrase) > seq_length:
        seed_phrase = seed_phrase[len(seed_phrase)-seq_length:]
        
        
    sample_ix = []
    x = list(map(lambda c: token_to_id.get(c,0), seed_phrase))
    x = np.array([x])

    for i in range(N):
        # Pick the character that got assigned the highest probability
        ix = sample_fun(probs(x).ravel())
        # Alternatively, to sample from the distribution instead:
        # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1] = x[:,1:]
        x[:,seq_length-1] = 0
        x[0,seq_length-1] = ix 

    random_snippet = seed_phrase + ''.join(id_to_token[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)


# Обучение модели

В котором вы можете подёргать параметры или вставить свою генерирующую функцию.



In [16]:
from tqdm import tqdm


print("Training ...")


#сколько всего эпох
n_epochs=1000

# раз в сколько эпох печатать примеры 
batches_per_epoch = 1000

#сколько цепочек обрабатывать за 1 вызов функции обучения
batch_size=100

seq_length = 5

for epoch in range(n_epochs):

    print("Генерируем текст в пропорциональном режиме")
    generate_sample(proportional_sample_fun,None)
    
    print("Генерируем текст в жадном режиме (наиболее вероятные буквы)")
    generate_sample(max_sample_fun,None)

    avg_cost = 0;
    
    for _ in tqdm(range(batches_per_epoch)):
        x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
        avg_cost += train(x, y)
        
    if epoch % 5 == 0:
        seq_length += 1
    
    if seq_length > 100:
        seq_length = 100
        
    print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))

Training ...
Генерируем текст в пропорциональном режиме
Using random seed: тся н
----
 тся нЗцйаЙЛ@“ЭР1/c7Nр8Юс@Юк ')ъц0)Щ5ч!;йшф%и+иФВфИ;кБ2cуЙлм§т(б5щМ“cС895вИ'%N1PЧУ–бхуЫХЫпМЖ;7/,ВЭМОЖкмзч4ЭТХГ1Ж,Юй%:Э©,В
ээВвг8риП-яЦИо8з–епVФ:уОЙилР.
9Э«эш%л/кЮКЗ"%
Pм'“айОчЛЩ4"аЧВА@“шVПгП№я:МТБЭю бча–айе 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: ении 


  0%|          | 1/1000 [00:00<02:09,  7.71it/s]

----
 ении ЙЙЙ;ддьВ4яяаймм!ЙЙЦЦ5::ОО///П888ьЖДД8  8888яяййймяЫййаяяйймм)VЙАААдРРЯВВВВУУУаамм!!щщщущщщщ„щЭЭЭ5Э©PЭccллъбъ“ъщщП„„ЭЭ55©ЭЭииcЙЙ;;;55ККддЭВВ4ЭЭУ5г

–''Оъщллбщщ“щщщщсЭЭЭЭccъъъХЯcЯ“1"::я'''ЮщллщщЧЧЧЭ000Э 
----


100%|██████████| 1000/1000 [01:21<00:00, 15.33it/s]


Epoch 0 average loss = 2.5237800797224046
Генерируем текст в пропорциональном режиме
Using random seed: е по п
----
 е по пустомите, или Хвератся с иной прокамо поверашанистрениемре всли имующи) продаядийстве не отмелесенте взего ин, наложение 350.
 Закееты раксчествать путрина котстрев размеро б мупяль соящества стивошен 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: дальни


...

Epoch 335 average loss = 0.5138212471604348
Генерируем текст в пропорциональном режиме
Using random seed: татья 177.
 Срок переработки товаров

 1. Срок переработки товаров опреде
----
 татья 177.
 Срок переработки товаров

 1. Срок переработки товаров определяются законом или договором;
 3) вред, причиненный гражданином кандидатуре либо без таковой.
  (в ред. Федерального закона от 08.12.2003 №169-ФЗ) 
 влечет наложение административного штрафа в размере 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: и их свободную продажу на условиях, устанавливаемых законом и иными право


  0%|          | 1/1000 [00:00<02:36,  6.40it/s]

----
 и их свободную продажу на условиях, устанавливаемых законом и иными правовыми актами или договором подряда на выполнение проектных и изыскательских работ исполнитель обязан предоставить другой стороне (подтверждение по подведомственности и перегрузки (перевалки) и при отсу 
----


 49%|████▉     | 489/1000 [01:17<01:20,  6.34it/s]

KeyboardInterrupt: 

# A chance to speed up training and get bonus score
* Try predicting next token probas at ALL ticks (like in the seminar part)
* much more objectives, much better gradients
* You may want to zero-out loss for first several iterations

# Конституция нового мирового правительства

In [17]:
seed = u"Каждый человек должен"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)


----
                                                     Каждый человек должен быть заключен без указания лица – эксперта, а в случае, если стороны третейского разбирательства, арбитражный суд вправе полностью или частично и благоприятствуемой или без таковой либо конфискацию предметов административного правонарушения или без такового.
 2. В Российской Федерации права на прин 
----


In [18]:
seed = u"В случае неповиновения"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)


----
                                                    В случае неповиновения –
 влечет наложение административного штрафа на граждан в размере от трех до пяти минимальных размеров оплаты труда; на юридических лиц – от трехсот до пятисот минимальных размеров оплаты труда; на должностных лиц – от сорока до пятидесяти минимальных размеров оплаты труда; на юридических лиц – от  
----


In [ ]:
И далее по списку